In [33]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
import glob
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')


pd.set_option('display.max_rows', 200)

In [34]:
atlasData = pd.read_pickle('../ATLAS_data/CutFlow_Chargino.pcl')
atlasData['label'] = 'ATLAS'

In [35]:
files = {'10 ns' : '../Wino_0j_match/Events/run_88/wino_1300_10ns_atlas_2018_42.pcl'}

In [36]:
allData = atlasData
for label,f in files.items():
    recastData = pd.read_pickle(f)
    recastData['label'] = label
    columns = [c for c in atlasData.columns if c in recastData]
    recastData = recastData[columns]
    allData = pd.concat((allData,recastData))
    
allData.sort_values(['mLLP','tau_ns'],inplace=True)        

In [37]:
allData.T

,0,1,0
mLLP,1300.0,1300.0,1300.0
tau_ns,10.0,10.0,10.0
Total,18.11,1.0,"(20.8, 1.0)"
Event and jet Cleaning,NaN,NaN,"(20.6, 0.995)"
Trigger,10.46,0.5774,"(11.2, 0.539)"
Primary vertex,NaN,NaN,"(11.2, 0.539)"
$E_{T}^{miss}>170$ GeV,NaN,NaN,"(7.9, 0.381)"
Track in PV,NaN,NaN,"(7.8, 0.378)"
$p_{T} > 50$ GeV,NaN,NaN,"(6.8, 0.326)"
Track quality requirements,NaN,NaN,"(5.8, 0.279)"


In [38]:
recastData = pd.read_pickle(f)
recastData.T

,0,1
Luminosity (1/fb),139.0,139.0
mLLP,1300.0,1300.0
mLSP,1299.0,1299.0
width,0.0,0.0
tau_ns,10.0,10.0
Total MC Events,68020,68020
Total xsec-pTcut (pb),0.00013,0.00013
Total xsec (pb),0.000261,0.000261
Total,18.11,1.0
Trigger,10.46,0.5774
